- Utilisez le prétraitement effectuer sur le vin
- Adapter la target (ŷ) à la modèlisation d’un problème de classification
- Créer un réseau de neurones et entrainez le sur votre jeu de donnée.
- Évaluer ses prédictions

In [11]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Sequential

In [6]:
url = "https://gist.githubusercontent.com/jsz4n/b7ca11015784086788022a539935d0cf/raw/a8c3abf0a31f5c0df5e0ddd76fb9b289bac9bed1/winequality-red.csv"
df = pd.read_csv(url,sep =";")

In [7]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [37]:
X = df.copy()
y = pd.DataFrame(X.pop("quality"))

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Prétraitement

- Normalisation du jeu de donnéës
- OHE de la target

In [39]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [40]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(y_train)
y_train = ohe.transform(y_train)*1.
y_test = ohe.transform(y_test)*1.

## Construction du réseau de neurones

In [41]:
X.shape, y_train.shape

((1599, 11), (1279, 6))

In [42]:
model = Sequential([
    layers.Input(shape=(11,)),
    layers.Dense(30),
    layers.Dense(15),
    layers.Dense(6,name = "output"),
    layers.Softmax()
])

In [43]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 30)             │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │           465 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 6)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_2 (Softmax)             │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 921 (3.60 KB)

 Trainable params: 921 (3.60 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy", "precision"])

## Entrainement du modèle

In [45]:
model.fit(X_train_scaled, y_train, epochs=30, validation_split=.1)

Epoch 1/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.2176 - loss: 1.9003 - precision: 0.1560 - val_accuracy: 0.4219 - val_loss: 1.5943 - val_precision: 0.3846
Epoch 2/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4986 - loss: 1.4504 - precision: 0.7675 - val_accuracy: 0.5469 - val_loss: 1.3773 - val_precision: 0.6250
Epoch 3/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5616 - loss: 1.2940 - precision: 0.6964 - val_accuracy: 0.5938 - val_loss: 1.2383 - val_precision: 0.6250
Epoch 4/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5771 - loss: 1.1621 - precision: 0.7275 - val_accuracy: 0.6172 - val_loss: 1.1308 - val_precision: 0.6316
Epoch 5/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5869 - loss: 1.1069 - precision: 0.6903 - val_accuracy: 0.6094 - val_loss: 1.0696 - val_precision: 0.6176
Epoch 6/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5913 - loss: 1.0146 - precision: 0.6865 - val_accuracy: 0.5859 - val_loss: 1.0240 - v

## Evaluation

In [63]:
y_pred = model.predict(X_test_scaled)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [69]:
ohe.get_metadata_routing()

{}

In [66]:
tf.argmax(y_pred,1)

<tf.Tensor: shape=(320,), dtype=int64, numpy=
array([2, 2, 3, 2, 3, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2,
       3, 3, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 2, 2, 3, 2,
       3, 3, 4, 2, 2, 3, 2, 2, 3, 3, 2, 2, 3, 2, 3, 2, 2, 3, 2, 2, 4, 2,
       4, 2, 3, 2, 4, 2, 3, 3, 3, 2, 4, 3, 3, 4, 2, 4, 2, 3, 3, 4, 2, 3,
       3, 2, 3, 2, 3, 3, 2, 3, 2, 3, 2, 3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 2,
       4, 2, 3, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2,
       2, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 2, 3, 3, 2, 3,
       3, 3, 2, 3, 2, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3,
       3, 2, 2, 2, 2, 3, 2, 4, 2, 3, 2, 3, 4, 2, 3, 3, 3, 4, 3, 2, 2, 2,
       2, 3, 2, 2, 2, 2, 4, 3, 2, 3, 3, 3, 3, 2, 3, 4, 4, 3, 2, 2, 3, 2,
       2, 3, 3, 3, 2, 2, 2, 4, 2, 2, 2, 2, 3, 3, 3, 3, 2, 3, 2, 2, 3, 2,
       3, 3, 2, 2, 3, 2, 4, 2, 3, 3, 2, 2, 2, 2, 3, 3, 3, 4, 3, 3, 2, 4,
       3, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2, 4, 2, 2, 2, 2, 3,
     

In [50]:
model.evaluate(X_test_scaled, y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5773 - loss: 0.9417 - precision: 0.6040


[0.9363806843757629, 0.574999988079071, 0.6047430634498596]